In [20]:
from dotenv import load_dotenv
import os
import openai
import pandas as pd
import pylibmagic
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [2]:
load_dotenv(verbose=True)
dotenv_path = os.path.join("../.env")
load_dotenv(dotenv_path)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# データフォルダとChromaのキャッシュフォルダのパス
novel_file_path = "../data/novels/"
chroma_cache_dir = "../.chroma_cache/"

In [4]:
# Chroma DBの永続化処理
if not os.path.exists(chroma_cache_dir):
    os.makedirs(chroma_cache_dir)

# Chromaデータベースの永続化パス
persist_directory = chroma_cache_dir

In [5]:
# 小説データをロード
loader = DirectoryLoader(
    novel_file_path,
    glob="*.txt"
)
documents = loader.load()

In [7]:
# テキストを分割 (長いテキストを適切に扱うために分割)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=200,
)
texts = text_splitter.split_documents(documents)

# OpenAIエンベディングを使ってテキストをエンベディングに変換
embeddings = OpenAIEmbeddings(
    openai_api_key=openai.api_key
)

# Chromaが既に永続化されているか確認
if os.path.exists(os.path.join(persist_directory, "index")):
    # 永続化されたDBをロード
    vectorstore = Chroma(
        persist_directory=persist_directory, embedding_function=embeddings
        )
    print("Chroma DBをキャッシュからロードしました。")
else:
    # 初回実行: Chroma DBにデータを格納
    vectorstore = Chroma.from_documents(
        texts,
        embeddings, persist_directory=persist_directory
    )
    vectorstore.persist()  # 永続化
    print("Chroma DBを新たに作成し、永続化しました。")


Chroma DBを新たに作成し、永続化しました。


/var/folders/5_/6wgs064d6yj6tsvtkjt8g_740000gn/T/ipykernel_30237/260691972.py:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()  # 永続化


In [17]:
# 作成したVectorStoreからの検索を設定
retriever = vectorstore.as_retriever()

# OpenAI LLMの設定 (RAGの生成部分)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=openai.api_key
)

# 検索と生成を統合したチェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 生成部分で関連情報を補完
    retriever=retriever,
    return_source_documents=True  # 検索に使用されたソースドキュメントを返す
)

In [19]:
# 質問を投げて、RAGの結果を取得
query = "「死生に関するいくつかの断想」で最初に出てくる日付を答えてください"
result = qa_chain.invoke(query)

# 結果を表示
print(result)

{'query': '「死生に関するいくつかの断想」で最初に出てくる日付を答えてください', 'result': '最初に出てくる日付は「七月二五日」です。', 'source_documents': [Document(metadata={'source': '../data/novels/7.txt'}, page_content='死生に関するいくつかの断想 BITS OF LIFE AND DEATH 小泉八雲\u3000Lafcadio Hearn 林田清明訳\n\n------------------------------------------------------\n\n【テキスト中に現れる記号について】\n\n《》：ルビ\n\n（例）御幣《ごへい》\n\n｜：ルビの付く文字列の始まりを特定する記号\n\n（例）杉本｜嘉作《かさく》\n\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\n\n（例）［＃１９字下げ］\n\n-------------------------------------------------------\n\n［＃１９字下げ］１［＃「１」は中見出し］\n\n七月二五日。\u3000今週は思いがけない訪問が三つ、わが家にあった。\n\n最初のものは、井戸掃除職人たちだった。毎年すべての井戸は空にされ、掃除され、井戸の神様である水神様が荒れ狂わないようにしなければならない。この時に、私は、日本の井戸と、ミズハノメノミコト（水波能売命）とも呼ばれる二つの名を持つ井戸の守り神にまつわるいくつかの事柄を知ったのだった。\n\n水神様は、屋敷の持ち主が浄めについてのきまりをしっかり守っていれば、井戸の水を甘露にして、かつ冷たく保って、あらゆる井戸を守ってくれる。これらの掟を破った者には病や死が訪れるという。稀には、この神は蛇の姿となって現れることがある。私はこの神を祀る神社を一度も見たことがない。しかし、毎月一度は、近所の神主が井戸のある敬虔な家庭を訪れて、井戸の神様に古式の祈りを捧げ、幟や紙の御幣《ごへい》を井戸の端に立てるのである。井戸が清掃された後にも、また、これがなされる。新しい水の最初の一汲みは男たちがしなければならない。というのは、女が最初に汲めば、その井戸はそれ以後ずっと濁ったままであるからだという。

In [30]:
result["source_documents"][0].page_content

'死生に関するいくつかの断想 BITS OF LIFE AND DEATH 小泉八雲\u3000Lafcadio Hearn 林田清明訳\n\n------------------------------------------------------\n\n【テキスト中に現れる記号について】\n\n《》：ルビ\n\n（例）御幣《ごへい》\n\n｜：ルビの付く文字列の始まりを特定する記号\n\n（例）杉本｜嘉作《かさく》\n\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\n\n（例）［＃１９字下げ］\n\n-------------------------------------------------------\n\n［＃１９字下げ］１［＃「１」は中見出し］\n\n七月二五日。\u3000今週は思いがけない訪問が三つ、わが家にあった。\n\n最初のものは、井戸掃除職人たちだった。毎年すべての井戸は空にされ、掃除され、井戸の神様である水神様が荒れ狂わないようにしなければならない。この時に、私は、日本の井戸と、ミズハノメノミコト（水波能売命）とも呼ばれる二つの名を持つ井戸の守り神にまつわるいくつかの事柄を知ったのだった。\n\n水神様は、屋敷の持ち主が浄めについてのきまりをしっかり守っていれば、井戸の水を甘露にして、かつ冷たく保って、あらゆる井戸を守ってくれる。これらの掟を破った者には病や死が訪れるという。稀には、この神は蛇の姿となって現れることがある。私はこの神を祀る神社を一度も見たことがない。しかし、毎月一度は、近所の神主が井戸のある敬虔な家庭を訪れて、井戸の神様に古式の祈りを捧げ、幟や紙の御幣《ごへい》を井戸の端に立てるのである。井戸が清掃された後にも、また、これがなされる。新しい水の最初の一汲みは男たちがしなければならない。というのは、女が最初に汲めば、その井戸はそれ以後ずっと濁ったままであるからだという。\n\n水神様の仕事を手助けする使者《おつかい》はほとんどいない。ただ、フナ（１）［＃「（１）」は行右小書き］という小さな魚がいる。一匹か二匹のフナがどの井戸にもいて、幼虫から水を綺麗にする。井戸浚いのとき、この魚は大事にされる。私の井戸にも一組のフナがいることを知ったのも、井戸浚い職人たちが来たこのときであった。井戸水が溜められている

In [22]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../data/query.csv", encoding="utf-8")

In [31]:
# 各問題に対して回答と証拠を取得する関数
def get_answer_and_evidence(problem):
    result = qa_chain.invoke(problem)
    answer = result["result"]  # 回答部分
    evidence = result["source_documents"][0].page_content # 証拠部分を抽出
    return answer, evidence

# 各行の問題に対して処理を実行し、回答と証拠を取得
answers = []
evidences = []

for _, row in query_df.iterrows():
    problem = row['problem']
    answer, evidence = get_answer_and_evidence(problem)
    answers.append(answer)
    evidences.append(evidence)

# DataFrameに回答と証拠を追加
query_df['answer'] = answers
query_df['evidence'] = evidences

In [35]:
query_df["index"] = query_df.index

In [36]:
# 結果をCSVファイルとして保存
query_df.to_csv("../result/output_with_answers_and_evidence_20240912.csv", index=False)

In [40]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../data/evaluation/submit_20240912/predictions.csv",
    index=False,
    header=False
)